# Pedestrian Tracking across frames of video

### This notebook will use the object identification from the Pedestrian detection notebook 
- A selected video will be broken into frames
- The pedestrians will be identified and logged
- coordinates of bounding box will be tracked for 6 frames
- Algorithim will try to predict where the pedestrian will be 6 frames from the current frame

In [1]:
import os
import sys
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print("Root Directory: ", ROOT_DIR)
sys.path.append(ROOT_DIR)

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from scripts import coco
from scripts import display_instance as di

Root Directory:  /home/sam/Documents/GitHub/honoursProject/PedestrianDetectionSGall


In [2]:
vid_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/video/"
frames_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/frames/"
output_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/output_vid/"
final_imgs_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/output_img/"

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

In [4]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "weights/mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                'bus', 'train', 'truck', 'boat', 'traffic light',
                'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                'kite', 'baseball bat', 'baseball glove', 'skateboard',
                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                'teddy bear', 'hair drier', 'toothbrush']

config = InferenceConfig()

In [5]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

2022-03-10 14:31:20.293485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 14:31:20.421849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 14:31:20.422197: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 14:31:20.423736: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Instructions for updating:
Use fn_output_signature instead


In [6]:
# Get video file name 

def get_video_name(vid_dir):
    files = os.listdir(vid_dir) 
    s = ""
    for file in files: 
        str_name = file[:21]
        s = str_name
    return s

vid_name = get_video_name(vid_dir)

video_path = vid_dir + vid_name
print(video_path)

/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/video/cadf77c3-2857db38.mov


##### Run video to confirm detection

##### Takes about 4 mins

cap = cv2.VideoCapture(video_path)
while cap.isOpened():
    ret, image = cap.read()
    results = model.detect([image], verbose=1)
    r = results[0]
    image = di.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
    cv2.imshow('MaskRCNN Object Detection/Instance Segmentation', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [7]:
# Break video down into frames

cap = cv2.VideoCapture(video_path)
i = 0
while cap.isOpened():
    ret, image = cap.read()
    if ret == False:
        break
    cv2.imwrite(os.path.join(frames_dir, vid_name + str(i) + ".jpg"), image)
    i += 1
cap.release()
cv2.destroyAllWindows()

In [13]:
# Create collection for frame names in frames_dir

def get_frame_list(frames_dir):
    files = os.listdir(frames_dir) 
    s = []
    for file in files: 
        str_name = file[:29]
        s.append(str_name) 
    return s

frames_list = get_frame_list(frames_dir)

In [15]:
# Read first frame and locate pedestrians

for frame in frames_list:
    processed_frames = []
    frame_location = frames_dir + frame
    cap = cv2.imread(frame_location)
    while cap.isOpened():
        ret, image = cap.read()
        results = model.detect([image], verbose=1)
        r = results[0]
        image = di.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
        cv2.imshow('MaskRCNN Object Detection/Instance Segmentation', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
cv2.destroyAllWindows()




AttributeError: 'numpy.ndarray' object has no attribute 'isOpened'

In [ ]:
results = model.detect([frames_list], verbose=1)
r = results[0]
frame = di.display_instances(frames_list, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
cv2.imshow('MaskRCNN Object Detection/Instance Segmentation', frame)
